In [102]:
import pandas as pd
import seaborn as sns

df = pd.read_csv('../data/dataset.csv')
print("Dataset shape:", df.shape)
df.head()


Dataset shape: (614, 13)


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [103]:
# X_train = pd.read_csv('../data/X_train.csv').values
# X_test = pd.read_csv('../data/X_test.csv').values
# y_train = pd.read_csv('../data/y_train.csv')['Loan_Status'].values
# y_test = pd.read_csv('../data/y_test.csv')['Loan_Status'].values

In [104]:
# Fill categorical nulls with mode
for col in ['Gender', 'Married', 'Dependents', 'Self_Employed', 'Loan_Amount_Term', 'Credit_History']:
    df[col] = df[col].fillna(df[col].mode()[0])

# Fill numerical null
df['LoanAmount'] = df['LoanAmount'].fillna(df['LoanAmount'].mean())
df['LoanAmount_log'] = np.log(df['LoanAmount'])


# Confirm no null values remain
print("Null values left:\n", df.isnull().sum())


Null values left:
 Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
LoanAmount_log       0
dtype: int64


In [105]:
print(df.columns.tolist())
feature_columns = [
    'Gender',
    'Married',
    'Dependents',
    'Education',
    'Self_Employed',
    'ApplicantIncome',
    'CoapplicantIncome',
    'LoanAmount',
    'Loan_Amount_Term',
    'Credit_History',
    'Property_Area'
]

X = df[feature_columns].values
y = df['Loan_Status'].values
print("Feature shape:", X.shape)
print("Target shape:", y.shape)

['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status', 'LoanAmount_log']
Feature shape: (614, 11)
Target shape: (614,)


In [106]:
from sklearn.preprocessing import LabelEncoder

# Categorical columns to encode
categorical_cols = ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Property_Area']

le = LabelEncoder()

for col in categorical_cols:
    df[col] = le.fit_transform(df[col].astype(str))

# Confirm encoding worked:
print(df[categorical_cols].head())


   Gender  Married  Dependents  Education  Self_Employed  Property_Area
0       1        0           0          0              0              2
1       1        1           1          0              0              0
2       1        1           0          0              1              2
3       1        1           0          1              0              2
4       1        0           0          0              0              2


In [107]:
X = df[feature_columns].values
y = df['Loan_Status'].values

In [108]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Train baseline model
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)
y_pred = nb_model.predict(X_test)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Baseline Naive Bayes Accuracy: {accuracy:.4f}")


Baseline Naive Bayes Accuracy: 0.8455


In [109]:
# Sensitive feature: Gender (already encoded)
sensitive_feature = X_test[:, feature_columns.index('Gender')]  # extracts Gender column from X_test

print("Unique Gender values in test set:", np.unique(sensitive_feature))


Unique Gender values in test set: [0. 1.]


In [113]:
from sklearn.metrics import accuracy_score
from fairlearn.metrics import MetricFrame, demographic_parity_difference

# Grouped fairness analysis
mf = MetricFrame(
    metrics=accuracy_score,
    y_true=y_test,
    y_pred=y_pred,
    sensitive_features=sensitive_feature
)

print("Accuracy by Gender Group:\n", mf.by_group)
print("Overall Accuracy:", mf.overall)

dpd = demographic_parity_difference(
    y_test, y_pred, sensitive_features=sensitive_feature
)
print("Demographic Parity Difference:", dpd)


Accuracy by Gender Group:
 sensitive_feature_0
0.0    0.840000
1.0    0.846939
Name: accuracy_score, dtype: float64
Overall Accuracy: 0.8455284552845529
Demographic Parity Difference: 0.09632653061224494


In [112]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score
from fairlearn.metrics import (
    demographic_parity_difference,
    equalized_odds_difference,
    selection_rate,
    false_positive_rate,
    false_negative_rate,
    MetricFrame
)

# ✅ Ensure your y_test and y_pred are properly encoded to [0, 1]
# Convert Loan_Status from 'Y'/'N' to 1/0 if not already done:
y_test = np.where(y_test == 'Y', 1, 0) if y_test.dtype.kind in {'U', 'O'} else y_test
y_pred = np.where(y_pred == 'Y', 1, 0) if y_pred.dtype.kind in {'U', 'O'} else y_pred

print(np.unique(y_test), "✅ y_test unique values (should be 0 and 1)")
print(np.unique(y_pred), "✅ y_pred unique values (should be 0 and 1)")

# ✅ sensitive_feature already selected correctly (Gender column from X_test):
# Example if not already defined:
# sensitive_feature = X_test[:, feature_columns.index('Gender')]

print("🔍 COMPREHENSIVE FAIRNESS ANALYSIS")
print("=" * 50)

# 1. Demographic Parity Difference
dpd = demographic_parity_difference(y_test, y_pred, sensitive_features=sensitive_feature)
print(f"📊 Demographic Parity Difference: {dpd:.4f}")

# 2. Equalized Odds Difference
eod = equalized_odds_difference(y_test, y_pred, sensitive_features=sensitive_feature)
print(f"⚖️  Equalized Odds Difference: {eod:.4f}")

# 3. Selection Rate by Group
mf_selection = MetricFrame(
    metrics=selection_rate,
    y_true=y_test,
    y_pred=y_pred,
    sensitive_features=sensitive_feature
)
print(f"📈 Selection Rate by Gender:\n{mf_selection.by_group}")

# 4. False Positive Rate by Group
mf_fpr = MetricFrame(
    metrics=false_positive_rate,
    y_true=y_test,
    y_pred=y_pred,
    sensitive_features=sensitive_feature
)
print(f"🚨 False Positive Rate by Gender:\n{mf_fpr.by_group}")

# 5. False Negative Rate by Group
mf_fnr = MetricFrame(
    metrics=false_negative_rate,
    y_true=y_test,
    y_pred=y_pred,
    sensitive_features=sensitive_feature
)
print(f"🚨 False Negative Rate by Gender:\n{mf_fnr.by_group}")

# 6. Precision and Recall by Group
mf_precision = MetricFrame(
    metrics=precision_score,
    y_true=y_test,
    y_pred=y_pred,
    sensitive_features=sensitive_feature
)
mf_recall = MetricFrame(
    metrics=recall_score,
    y_true=y_test,
    y_pred=y_pred,
    sensitive_features=sensitive_feature
)
print(f"🎯 Precision by Gender:\n{mf_precision.by_group}")
print(f"🎯 Recall by Gender:\n{mf_recall.by_group}")

# ✅ Final Summary Table
fairness_summary = pd.DataFrame({
    'Metric': ['Demographic Parity Diff', 'Equalized Odds Diff'],
    'Value': [dpd, eod],
    'Interpretation': [
        'Good' if abs(dpd) < 0.1 else 'Needs Attention',
        'Good' if abs(eod) < 0.1 else 'Needs Attention'
    ]
})
print("\n📋 FAIRNESS SUMMARY TABLE")
print("=" * 50)
print(fairness_summary)


[0 1] ✅ y_test unique values (should be 0 and 1)
[0 1] ✅ y_pred unique values (should be 0 and 1)
🔍 COMPREHENSIVE FAIRNESS ANALYSIS
📊 Demographic Parity Difference: 0.0963
⚖️  Equalized Odds Difference: 0.0808
📈 Selection Rate by Gender:
sensitive_feature_0
0.0    0.720000
1.0    0.816327
Name: selection_rate, dtype: float64
🚨 False Positive Rate by Gender:
sensitive_feature_0
0.0    0.363636
1.0    0.444444
Name: false_positive_rate, dtype: float64
🚨 False Negative Rate by Gender:
sensitive_feature_0
0.0    0.000000
1.0    0.042254
Name: false_negative_rate, dtype: float64
🎯 Precision by Gender:
sensitive_feature_0
0.0    0.777778
1.0    0.850000
Name: precision_score, dtype: float64
🎯 Recall by Gender:
sensitive_feature_0
0.0    1.000000
1.0    0.957746
Name: recall_score, dtype: float64

📋 FAIRNESS SUMMARY TABLE
                    Metric     Value Interpretation
0  Demographic Parity Diff  0.096327           Good
1      Equalized Odds Diff  0.080808           Good
